# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [3]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# webdrivers
# firefox -> geckodriver
# chrome -> chromedriver

In [7]:
from selenium import webdriver

# Step 1: Open the browser
browser = webdriver.Chrome()



In [6]:
import pandas as pd

# Step 2: Import database
table = pd.read_excel("commodities.xlsx")
display(table)



,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


In [13]:
from unidecode import unidecode

# Step 3: Search the current price for each product in the database
for row in table.index:
    product = table.loc[row, "Produto"]
    
    # remove accents
    product = unidecode(product)

    link = f"https://www.melhorcambio.com/{product}-hoje"
    browser.get(link) # navigate to link

    element = browser.find_element("id", "comercial") # gets the price HTML element
    
    # .click() -> click
    # .send_keys("text") -> write
    # .get_attribute() -> gets element attribute
    
    price = element.get_attribute("value") # gets the price value
    
    price = float(price.replace(".", "").replace(",", ".")) # format price
    
    # Step 4: Store current prices in the database
    table.loc[row, "Preço Atual"] = price

display(table)


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,NaN
1,Soja,163.59,162.64,NaN
2,Boi,282.20,276.05,NaN
3,Petróleo,424.37,391.91,NaN
4,Algodão,497.76,498.88,NaN
5,Açúcar,136.23,134.64,NaN
6,Café,1092.87,1087.87,NaN
7,Ouro,321.77,324.51,NaN
8,Trigo,1549.11,1546.08,NaN
9,Tilápia,9.05,9.05,NaN


In [14]:
# Step 5: Indicate which products should be bought (current price is lower than the target price)
table["Comprar"] = table["Preço Atual"] < table["Preço Ideal"]
display(table)


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,85.49,False
1,Soja,163.59,162.64,True
2,Boi,282.20,276.05,True
3,Petróleo,424.37,391.91,True
4,Algodão,497.76,498.88,False
5,Açúcar,136.23,134.64,True
6,Café,1092.87,1087.87,True
7,Ouro,321.77,324.51,False
8,Trigo,1549.11,1546.08,True
9,Tilápia,9.05,9.05,False


In [15]:
# Step 6: Export new database to excel
table.to_excel("commodities_updated.xlsx", index=False)

In [ ]:
# closes browser
browser.quit()